In [ ]:
# Jinseok Ryu, Ph.D.
# jinseuk56@gmail.com
# 20230927

from abtem import (GridScan, PixelatedDetector, Potential,
                   Probe, show_atoms, SMatrix, AnnularDetector, FrozenPhonons)
from abtem.measure import Measurement, Calibration, bandlimit, center_of_mass
from abtem.utils import energy2wavelength
from abtem.structures import orthogonalize_cell
from abtem.transfer import CTF, scherzer_defocus
from ase.io import read
from ase.build import mx2, surface
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.patches import Rectangle
from pylab import cm
from matplotlib.widgets import RectangleSelector
import matplotlib.patches as pch
import tifffile
import tkinter.filedialog as tkf
import json


device="cpu"
#device="gpu"

In [ ]:
unit_cell  = read(tkf.askopenfilename())
atoms_init = surface(unit_cell, (0,0,1), 1, periodic=True)
atoms_init = atoms_init * (1, 1, 2)


fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

show_atoms(atoms_init, ax = ax1, title = 'Beam view')
show_atoms(atoms_init, ax = ax2, plane = 'yz', title = 'Side view')
show_atoms(atoms_init, ax = ax3, plane = 'xz', title = 'Side view')

fig.tight_layout()
plt.show()

In [ ]:
num_atoms = atoms_init.numbers.shape[0]
elements = np.unique(atoms_init.numbers)
print(num_atoms)
print(elements)

atoms = atoms_init.copy()

In [ ]:
# vacancy generation
#def_element = elements
def_element = [74]
ind_element = []

for e in def_element:
    tmp_ind = np.where(atoms.numbers==e)
    ind_element.extend(tmp_ind[0].tolist())

def_ratio = 0.05
ri = np.random.choice(ind_element, int(len(ind_element)*def_ratio), replace=False)
print(ri)

for ai in ri:
    del atoms[ai]
    
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
show_atoms(atoms, ax = ax1, title = 'Beam view')
show_atoms(atoms, ax = ax2, plane = 'yz', title = 'Side view')
show_atoms(atoms, ax = ax3, plane = 'xz', title = 'Side view')
fig.tight_layout()
plt.show()

In [ ]:
# substitutional defect generation
def_element = [74]
sub_element = [42]
sub_element_prob = [1.0]
ind_element = []

for e in def_element:
    tmp_ind = np.where(atoms.numbers==e)
    ind_element.extend(tmp_ind[0].tolist())
    
def_ratio = 0.1
ri = np.random.choice(ind_element, int(len(ind_element)*def_ratio), replace=False)
print(ri)

for ai in ri:
    atoms.numbers[ai] = np.random.choice(sub_element, 1, p=sub_element_prob)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

show_atoms(atoms, ax = ax1, title = 'Beam view')
show_atoms(atoms, ax = ax2, plane = 'yz', title = 'Side view')
show_atoms(atoms, ax = ax3, plane = 'xz', title = 'Side view')

fig.tight_layout()
plt.show()

In [ ]:
# interstitial defect generation
# ongoing...

In [ ]:
frozen_phonons = FrozenPhonons(atoms, 16, sigmas=0.01, seed=1)
tds_potential = Potential(frozen_phonons, 
                      sampling=.05,
                      projection='infinite', 
                      slice_thickness=1, 
                      parametrization='kirkland',
                    device=device).build()
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
tds_potential.project().show(ax=ax, cmap="inferno")
plt.show()

In [ ]:
energy = 80E3
C3 = 1E3 # 100 nm
semiangle = 30 # mrad
sch_defocus = scherzer_defocus(C3, energy)
print(sch_defocus)

In [ ]:
#ctf = CTF(parameters={'C10': -75,'C12': 20, 'phi12': 0.785,'C30':-7e4}, semiangle_cutoff=semiangle)
ctf = CTF(parameters={'C10': sch_defocus*1.1, 'C30':C3}, semiangle_cutoff=semiangle)

probe = Probe(semiangle_cutoff=semiangle, energy=energy, ctf=ctf, device=device)
probe.grid.match(tds_potential)

print(probe.ctf.nyquist_sampling)

angle_step_default = (probe.wavelength *1000 / probe.extent[0], probe.wavelength *1000 / probe.extent[1])
print("angle step x", probe.wavelength *1000 / probe.extent[0])
print("angle step y", probe.wavelength *1000 / probe.extent[1])

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
probe.show(ax=ax[0])
probe.show(ax=ax[1], power=0.5)
plt.show()

In [ ]:
probe_step = 0.5 # Å
gridscan = GridScan((0,0), np.array(tds_potential.extent), sampling=probe_step)
print(gridscan.gpts)

In [ ]:
DP_shape = [128, 128] # height, width
detect_max_angle = semiangle*4
angle_pixel_size = (detect_max_angle*2/DP_shape[1], detect_max_angle*2/DP_shape[0]) # (mrad, mrad) tuple
print(angle_pixel_size)
detector = PixelatedDetector(max_angle=detect_max_angle, resample=angle_pixel_size)

In [ ]:
measurement = probe.scan(gridscan, [detector], tds_potential)
print(measurement.shape)

In [ ]:
print(measurement.shape)
print(*measurement.calibration_limits, sep="\n")
for i in range(measurement.dimensions):
    print(measurement.calibrations[i].name, measurement.calibrations[i].units,
          measurement.calibrations[i].sampling)

In [ ]:
measurement_sampling = tuple(energy2wavelength(energy)*1000/(cal.sampling * pixels) 
                                                      for cal,pixels 
                                                      in zip(measurement.calibrations[-2:], 
                                                             measurement.shape[-2:]))

print(f'pixelated_measurement sampling: {measurement_sampling} Å')

In [ ]:
measurement_extent = tuple(sampling*pixels for sampling,pixels 
                                      in zip(measurement_sampling, measurement.shape[-2:]))

print(f'pixelated_measurement extent: {measurement_extent} Å')

In [ ]:
bright_detector = AnnularDetector(inner=10, outer=semiangle)
bright_measurement = bright_detector.integrate(measurement)

maadf_detector = AnnularDetector(inner=semiangle, outer=detect_max_angle)
maadf_measurement = maadf_detector.integrate(measurement)

pacbed = measurement.mean(axis=(0, 1))

fig, ax = plt.subplots(2, 2, figsize=(10, 10))
measurement.show(ax=ax[0, 0], cmap="inferno", power=0.5)
pacbed.show(ax=ax[0, 1], cmap="inferno", power=0.5)
bright_measurement.show(ax=ax[1, 0])
maadf_measurement.show(ax=ax[1, 1])
fig.tight_layout()
plt.show()

In [ ]:
save_name = "output_file_name"
tifffile.imwrite(save_name+".tif", measurement.array)

In [ ]:
calibration_info = {}
calibration_info["material"] = "material_name"
calibration_info["beam energy"] = energy
calibration_info["convergence angle"] = semiangle
calibration_info["scan pixel size"] = (measurement.calibrations[0].sampling, measurement.calibrations[1].sampling)
calibration_info["angle pixel size"] = (measurement.calibrations[2].sampling, measurement.calibrations[3].sampling)

In [ ]:
with open(save_name+".txt", 'w') as file:
     file.write(json.dumps(calibration_info, ensure_ascii=False))